In [1]:
from pandas import DataFrame,Series
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 
import time

In [11]:
import sqlite3

# Abrimos la conexión con la BBDD
conn = sqlite3.connect("D:/master/Master-in-Data-Science/Proyecto/Datos/worldTimeSeries.sqlite")
cursor = conn.cursor()

# Vamos a intentar automatizar el análisis anterior, es decir, considerar los datos a partir de cierto año y a 
# continuacion filtrar por aquellas estaciones que tienen observaciones por año inferior a 364 dias
df_total = DataFrame()
continentes=["europa"]
#continentes=["europa","oceania","north_america"]

for cont in continentes:
    print("Inicio continente: " + cont + " a las " + str(time.strftime("%H:%M:%S")))
    # Se recuperan todas las estaciones del continentes que existían previamente a 1900 y existen en 2016
    rows = cursor.execute("select distinct station from world_time_"+cont+" where date = '19000101';")
        
    station_list_1900 = []
    for r in rows:
        station_list_1900.append({"station":r[0]})
        
    if len(station_list_1900) > 0:
        rows = cursor.execute("select distinct station from world_time_"+cont+" where date = '20160401';")        
        station_list_2016 = []
        for r in rows:
            station_list_2016.append({"station":r[0]})

        station_list_1900 = DataFrame(station_list_1900)
        station_list_2016 = DataFrame(station_list_2016)

        # Nos quedamos con las estaciones que estan en ambas listas

        station_list = station_list_1900.merge(station_list_2016)
        station_list = station_list.reset_index()

        if len(station_list) > 0:
            # Se compone la lista de estaciones identificadas
            stats = ""
            for s in station_list.as_matrix():
                if stats != "":
                    stats = stats + ","
                stats = stats + "'" + s[1] + "'" 
                
            # De todas las estaciones anteriores, se recupera el número de observaciones no nulas por año
            rows = cursor.execute("""select year, station, count(*) as n_records
                                     from (
                                        select cast(substr(date,1,4) as integer) as year, 
                                               station 
                                        from world_time_"""+cont+"""
                                        where station in (""" + stats + """) and tmax is not null and tmin is not null
                                     )
                                     where year >= 1900 and year < 2016
                                     group by year, station
                                     order by station,year;""")
            
            # Se obtiene la lista de estaciones con todas las observaciones
            stats = ""
            current_station = ""
            prev_year = 0
            
            for r in rows:
                # Se identifica un cambio de estacion, ya sea la primera o no
                if current_station == "" or r[1] != current_station:
                    if current_station != "": # Si no se trata de la primera iteracion del bucle
                        # Solo se guarda la estacion si tiene todas las observaciones y la ultima observacion es 2015
                        if todas_observaciones and prev_year == 2015:
                            if stats != "":
                                stats = stats + ","
                            stats = stats + "'" + current_station + "'"

                    # Reinicializacion de las variables de control
                    prev_year = 1899
                    current_station = r[1]
                    todas_observaciones = True
                    # Si la primera observacion no es la del año 1900 mas menos 1 año, entonces falta informacion
                    if r[0] - 1900 >= 3:
                        todas_observaciones = False

                # Si el numero de observaciones es inferior a 364 y faltan 2 o mas años
                if todas_observaciones and (r[2] < 360 or r[0] - prev_year >= 3):
                    todas_observaciones = False # La estacion no tiene todas las observaciones en el año
                
                prev_year = r[0]

            # Ultima estacion recuperada
            if todas_observaciones:
                if stats != "":
                    stats = stats + ","
                stats = stats + "'" + current_station + "'"

            # En este punto tenemos todas las estaciones con datos completos y sin huecos, recuperamos la informacion de
            # estas estaciones
            if stats != "":
                rows = cursor.execute("""
                    select 
                        a.year,
                        TMAX,
                        TMIN,
                        avg_TMAX,
                        avg_TMIN,
                        count(*) as n_records
                    from (
                        select cast(substr(date,1,4) as integer) as year, TMAX, TMIN
                        from world_time_"""+cont+""" 
                        where station in ("""+stats+""")
                        ) a 
                    left join (
                        select year,avg(TMAX) as avg_TMAX, avg(TMIN) as avg_TMIN
                        from(
                            select cast(substr(date,1,4) as integer) as year, TMAX, TMIN
                            from world_time_"""+cont+""" 
                            where station in ("""+stats+""")       
                            ) 
                            group by year
                        ) b
                    on a.year = b.year
                    where a.year >= 1900;""")                

                result=[]           
                for r in rows:
                    result.append({"year":r[0],"avg_tmax":r[1],"avg_tmin":r[2],"stdev_tmax":r[3],"stdev_tmin":r[4],"n_records":r[5]})

                df = DataFrame(result)    
                result = []
                                
                df['continent'] = cont
                df.to_csv("../datos/Resultado_calentamiento_1900_a.csv")
                    
    df.to_csv("../datos/Resultado_calentamiento_1900_a.csv")
conn.close()

Inicio continente: europa a las 20:40:24


In [ ]:
                rows = cursor.execute("""
                    select 
                        a.year,
                        avg(TMAX) as avg_tmax,
                        avg(TMIN) as avg_tmin,
                        sum((TMAX-avg_TMAX)) as stdev_tmax,
                        sum((TMIN-avg_TMIN)) as stdev_tmin,
                        count(*) as n_records
                    from (
                        select cast(substr(date,1,4) as integer) as year, TMAX, TMIN
                        from world_time_"""+cont+""" 
                        where station in ("""+stats+""")
                        ) a 
                    left join (
                        select year,avg(TMAX) as avg_TMAX, avg(TMIN) as avg_TMIN
                        from(
                            select cast(substr(date,1,4) as integer) as year, TMAX, TMIN
                            from world_time_"""+cont+""" 
                            where station in ("""+stats+""")       
                            ) group by year
                        ) b
                    on a.year = b.year and a.year >= 1900
                    group by a.year
                    order by a.year;""")                
